# How to pass additional parameters to tools

:::info Prerequisites

This guide assumes familiarity with the following concepts:
- [Chat models](/docs/concepts/#chat-models)
- [LangChain Tools](/docs/concepts/#tools)
- [How to use a model to call tools](/docs/how_to/tool_calling)
- [Chaining](/docs/how_to/sequence/)

:::

When using a chat model to call a tool in a chain or agent, it can be convenient for that tool to use information aside from arguments generated by the model.

This guide covers how to use a technique called [currying](https://en.wikipedia.org/wiki/Currying) to predefine certain parameters passed to a tool while letting the model generate the rest. It's similar in spirit to [partially applying variables to a prompt](/docs/how_to/prompts_partial/).

Let's mock out a function called `weather` as follows. Note that we aren't wrapping it with a `@tool` decorator:

In [1]:
def current_weather(location: str, use_cached_results: bool):
    """Use this to get the current weather in a given location."""
    if use_cached_results:
        # Use a cached response
        return f"The current weather in {location} is 50 degrees and rainy (cached)."
    else:
        # Use a freshly retrieved response
        return f"The current weather in {location} is 0 degrees and sunny."

You probably don't want the model to determine our caching strategy by populating `use_cached_results` - instead you probably want to define it yourself based on something like recency.

To do this, define a `curry` function that accepts a standard Python function, binding arguments to it and converting it into a tool:

In [2]:
import inspect
from functools import wraps

from langchain_core.tools import StructuredTool


def curry(func, **curried_kwargs):
    """Util to return partialed functions whose signatures also hide the curried variables."""

    @wraps(func)
    def wrapper(*args, **kwargs):
        # Merge curried kwargs with the new kwargs
        new_kwargs = {**curried_kwargs, **kwargs}
        return func(*args, **new_kwargs)

    # Get the original function's signature
    sig = inspect.signature(func)

    # Create a new signature without the curried parameters
    new_params = [p for name, p in sig.parameters.items() if name not in curried_kwargs]
    wrapper.__signature__ = sig.replace(parameters=new_params)

    return StructuredTool.from_function(wrapper)

You can wrap the `current_weather` function in the `curry` function to automatically set a `use_cached_result` to `True` and ensure that the model only generates a `location` parameter. Here's what that looks like when we bind it to a chat model:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="model" />
```

In [ ]:
# | output: false
# | echo: false

%pip install -qU langchain langchain_anthropic

import os
from getpass import getpass

from langchain_anthropic import ChatAnthropic

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass()

model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

In [9]:
curried_tool = curry(current_weather, use_cached_results=True)

llm_with_tools = model.bind_tools([curried_tool])

res = llm_with_tools.invoke("What is the weather in SF?")

res.tool_calls

[{'name': 'current_weather',
  'args': {'location': 'San Francisco'},
  'id': 'toolu_01KaLhdKqg8CvwCUS7Mmiom6'}]

You can see that the model does not generate a parameter for `use_cached_results`. If you then extend the chain to invoke the tool with the provided arguments, you can see that you'll always get a cached result rather than a fresh one:

In [10]:
# Simplified way to call the tool from model output
# Assumes the model always calls the weather tool
chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | curried_tool

chain.invoke("What is the weather in SF?")

'The current weather in San Francisco is 50 degrees and rainy (cached).'

## Next steps

You've now seen how to pass additional arguments to tools. Next, check out the following guides to learn more about tools:

- [Build agents using LangGraph](https://langchain-ai.github.io/langgraph/tutorials/introduction/)
- [Force a tool call](/docs/how_to/tool_choice)
- [Pass tool results back to model](/docs/how_to/tool_results_pass_to_model)

You can also check out the full list of how-to guides on tools:

- Building [tool-using chains and agents](/docs/how_to#tools)